In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import openpyxl  # openpyxl 모듈 추가
import re

sns.set_style('darkgrid')
sns.set_palette('Set3')
warnings.filterwarnings('ignore')
plt.rcParams["font.family"] = "NanumBarunGothic"


In [2]:

headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

from datetime import datetime, timedelta

# 시작일과 종료일 설정
start_day = datetime(2023, 1, 2)
end_day = datetime(2024, 4, 29)

# 시작일과 종료일 사이의 주count 계산
total_weeks = ((end_day - start_day).days // 7) + 1

# 엑셀 파일 생성
wb = openpyxl.Workbook()
ws = wb.active
ws.append(["Day", "Rank", "Title", "Album_img_urls","album_Code"])

# 주 단위로 날짜 출력
current_day = start_day
print(f"시작일 : {start_day}, 종료일 : {end_day}")
Week_Count = 0
   
while current_day <= end_day:
   # 주의 시작일과 끝일을 계산
   week_start = current_day.strftime("%Y%m%d")
   week_end = (current_day + timedelta(days=6)).strftime("%Y%m%d")
   current_day += timedelta(weeks=1)

   response = requests.get("https://www.melon.com/chart/week/index.htm?classCd=GN0000&moved=Y&startDay={0}&endDay={1}".format(week_start,week_end), headers=headers)
        
   URL = "https://www.melon.com/chart/week/index.htm?classCd=GN0000&moved=Y&startDay={0}&endDay={1}".format(week_start,week_end)
    
   print(URL)
   html = response.content.decode('utf-8','replace')
   soup = BeautifulSoup(html, 'html.parser',from_encoding='cp949')

   my_st = soup.select(
      '#frm > div > table > tbody'
   )
   
   for stock_name in my_st:
      # 앨범 코드 맬론 랭킹 1~50까지는 tr의 클래스를 lst50을 사용하고 51~100까지의 tr클래스는 lst100을 사용함
      Album_codes = []
      for tag in soup.find_all('tr'):
         if 'class' in tag.attrs:
            if 'lst50' in tag['class'] or 'lst100' in tag['class']:
                  Album_codes.append(tag['data-song-no'])
                  
      # 랭킹
      Rank = stock_name.select("td:nth-child(2) > div > span.rank")
      Rank_Num = [int(Rank.text) for Rank in Rank]

      # 제목
      Title = stock_name.select("td:nth-child(6) > div > div > div.ellipsis.rank01 > span > a")
      Title_Text = [Title.text for Title in Title]
      
      # 앨범 이미지
      Album_img_urls = [img['src'] for img in stock_name.select("td:nth-child(4) > div > a > img")]

      for rank, title_Text, album_img_urls, album_Code in zip(Rank_Num, Title_Text, Album_img_urls, Album_codes):
         # print(week_start, rank, name_text, prod_Text, album_img_urls, only_Codes)
         # print("====================================================")
         ws.append([week_start, rank, title_Text, album_img_urls, album_Code])

      # 주별 카운트 확인
      Week_Count += 1
      # 실행중인Count : 총Count
      print(f"Day_Count : {Week_Count}, Total_Count : {total_weeks}")
      
# 엑셀 파일 저장
file_path = r"C:\P_Project\1.Project\5-Project_data\1.Melon_Crawling_No_Detail_4.30.xlsx"
wb.save(file_path)
print("Excel 파일이 저장되었습니다:", file_path)

시작일 : 2023-01-02 00:00:00, 종료일 : 2024-04-29 00:00:00
https://www.melon.com/chart/week/index.htm?classCd=GN0000&moved=Y&startDay=20230102&endDay=20230108
Day_Count : 1, Total_Count : 70
https://www.melon.com/chart/week/index.htm?classCd=GN0000&moved=Y&startDay=20230109&endDay=20230115
Day_Count : 2, Total_Count : 70
https://www.melon.com/chart/week/index.htm?classCd=GN0000&moved=Y&startDay=20230116&endDay=20230122
Day_Count : 3, Total_Count : 70
https://www.melon.com/chart/week/index.htm?classCd=GN0000&moved=Y&startDay=20230123&endDay=20230129
Day_Count : 4, Total_Count : 70
https://www.melon.com/chart/week/index.htm?classCd=GN0000&moved=Y&startDay=20230130&endDay=20230205
Day_Count : 5, Total_Count : 70
https://www.melon.com/chart/week/index.htm?classCd=GN0000&moved=Y&startDay=20230206&endDay=20230212
Day_Count : 6, Total_Count : 70
https://www.melon.com/chart/week/index.htm?classCd=GN0000&moved=Y&startDay=20230213&endDay=20230219
Day_Count : 7, Total_Count : 70
https://www.melon.com/c